In [1]:
import pandas
import numpy

df_updrs_iii = pandas.read_csv('../source_data/MDS_UPDRS_Part_III.csv', low_memory=False)
df_patnos = pandas.read_csv('../source_data/non_hc_patnos.csv')
df_sigfall = pandas.read_csv('../source_data/sigfall_main.csv')
non_hc_patnos = list(df_patnos.values.flat)

In [2]:
df_updrs_iii = df_updrs_iii.query('PDSTATE != "OFF" ')
df_updrs_iii.shape

(15587, 55)

In [3]:
# --- Create dataframe that combines UPDRSiii data with SIGFALL & PREV_SIGFALL ---

def lookup_value(dframe,row,value):
    patno = row['PATNO']
    infodt = row['INFODT']
    
    s1 = dframe[(dframe.INFODT == infodt) & (dframe.PATNO == patno)][value]
    if s1.empty:    
        return numpy.nan
    else:
        return s1.values[0]
    
df_updrs_iii_sigfall_base = df_updrs_iii[df_updrs_iii['PATNO'].isin(non_hc_patnos)]
df_updrs_iii_sigfall_base = df_updrs_iii_sigfall_base.drop(['REC_ID','PAG_NAME', 'PDMEDDT',
       'PDMEDTM', 'PDSTATE', 'EXAMTM', 'DBS_STATUS','DYSKPRES', 'DYSKIRAT', 'PDTRTMNT','DBSONTM', 'DBSOFFTM','HRPOSTMED', 'HRDBSOFF', 'HRDBSON', 'ORIG_ENTRY',
       'LAST_UPDATE'], axis=1)
df_updrs_iii_sigfall_base['SIGFALL'] = df_updrs_iii_sigfall_base.apply (lambda row: lookup_value(df_sigfall,row,'SIGFALL'), axis=1)
df_updrs_iii_sigfall_base['PREV_SIGFALL'] = df_updrs_iii_sigfall_base.apply (lambda row: lookup_value(df_sigfall,row,'PREV_SIGFALL'), axis=1)
df_updrs_iii_sigfall_base = df_updrs_iii_sigfall_base.query('SIGFALL == 0.0 or SIGFALL == 1.0')
df_updrs_iii_sigfall_base = df_updrs_iii_sigfall_base.dropna(axis=0)


In [4]:
# convert columns to floats

updrs_qs = ['NP3SPCH', 'NP3FACXP',
       'NP3RIGN', 'NP3RIGRU', 'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL', 'NP3FTAPR',
       'NP3FTAPL', 'NP3HMOVR', 'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR',
       'NP3TTAPL', 'NP3LGAGR', 'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT',
       'NP3PSTBL', 'NP3POSTR', 'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR',
       'NP3KTRML', 'NP3RTARU', 'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ',
       'NP3RTCON', 'NP3TOT', 'NHY']

for x in updrs_qs:
    df_updrs_iii_sigfall_base[x] = df_updrs_iii_sigfall_base[x].astype(float)
    print(df_updrs_iii_sigfall_base[x].value_counts())

0.0    933
1.0    471
2.0    116
3.0     15
4.0      6
Name: NP3SPCH, dtype: int64
0.0    622
1.0    565
2.0    303
3.0     51
Name: NP3FACXP, dtype: int64
0.0    927
1.0    352
2.0    210
3.0     49
4.0      3
Name: NP3RIGN, dtype: int64
0.0    798
1.0    424
2.0    290
3.0     28
4.0      1
Name: NP3RIGRU, dtype: int64
0.0    889
1.0    388
2.0    236
3.0     28
Name: NP3RIGLU, dtype: int64
0.0    1036
1.0     295
2.0     174
3.0      35
4.0       1
Name: NP3RIGRL, dtype: int64
0.0    1056
1.0     267
2.0     180
3.0      38
Name: NP3RIGLL, dtype: int64
0.0    708
1.0    502
2.0    264
3.0     64
4.0      3
Name: NP3FTAPR, dtype: int64
0.0    671
1.0    491
2.0    289
3.0     88
4.0      2
Name: NP3FTAPL, dtype: int64
0.0    830
1.0    457
2.0    217
3.0     37
Name: NP3HMOVR, dtype: int64
0.0    763
1.0    478
2.0    249
3.0     50
4.0      1
Name: NP3HMOVL, dtype: int64
0.0    895
1.0    420
2.0    202
3.0     24
Name: NP3PRSPR, dtype: int64
0.0    824
1.0    467
2.0    207
3.0    

In [5]:
# Compute Delta values, from last sample event

def get_infodts(dframe,patno):
    return list(dframe[dframe.PATNO == patno]['INFODT'].values)

def date_to_tuple(date):
    year = date[-4:]
    month = date[:2]
    return (month,year,date)

def date_sort(dates):
    tup_list = map(date_to_tuple,dates)
    return sorted(tup_list, key = lambda x: (x[1],x[0]))

def get_index(date,dates):
    output = (False,999)    
    for idx, val in enumerate(dates):
        if val[2] == date:
            output = (True,idx)
    return output

def prev_value(dframe,row,q):
    output = numpy.nan
    patno = row['PATNO']
    infodt = row['INFODT']    
    dates = get_infodts(dframe,patno)
    sorted_dates = date_sort(dates)
    i = get_index(infodt,sorted_dates)
    
    if i[0] == True:
        try:
            prev_date_index = i[1] - 1
            if prev_date_index > -1:
                prev_date = sorted_dates[prev_date_index][2]
                output = dframe[(dframe.INFODT == prev_date ) & (dframe.PATNO == patno)][q].values[0]
        except:
            pass
    return output

def q_delta_actual(dframe,row,q):
    output = numpy.nan
    patno = row['PATNO']
    infodt = row['INFODT']
    
    current = dframe[(dframe.INFODT == infodt ) & (dframe.PATNO == patno)][q].values[0]
    prev = prev_value(dframe,row,q)
    
    try:
        output = current - prev
    except:
        pass
    return output

updrs_qs = ['NP3SPCH', 'NP3FACXP',
       'NP3RIGN', 'NP3RIGRU', 'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL', 'NP3FTAPR',
       'NP3FTAPL', 'NP3HMOVR', 'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR',
       'NP3TTAPL', 'NP3LGAGR', 'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT',
       'NP3PSTBL', 'NP3POSTR', 'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR',
       'NP3KTRML', 'NP3RTARU', 'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ',
       'NP3RTCON', 'NP3TOT', 'NHY']
dq_pairs = [(x,('D_'+x)) for x in updrs_qs]


for x in dq_pairs:
    df_updrs_iii_sigfall_base[x[1]] = df_updrs_iii_sigfall_base.apply (lambda row: q_delta_actual(df_updrs_iii_sigfall_base,row,x[0]), axis=1)


In [6]:
df_qs = df_updrs_iii_sigfall_base[['D_NP3SPCH', 'D_NP3FACXP',
       'D_NP3RIGN', 'D_NP3RIGRU', 'D_NP3RIGLU', 'D_NP3RIGRL', 'D_NP3RIGLL',
       'D_NP3FTAPR', 'D_NP3FTAPL', 'D_NP3HMOVR', 'D_NP3HMOVL', 'D_NP3PRSPR',
       'D_NP3PRSPL', 'D_NP3TTAPR', 'D_NP3TTAPL', 'D_NP3LGAGR', 'D_NP3LGAGL',
       'D_NP3RISNG', 'D_NP3GAIT', 'D_NP3FRZGT', 'D_NP3PSTBL', 'D_NP3POSTR',
       'D_NP3BRADY', 'D_NP3PTRMR', 'D_NP3PTRML', 'D_NP3KTRMR', 'D_NP3KTRML',
       'D_NP3RTARU', 'D_NP3RTALU', 'D_NP3RTARL', 'D_NP3RTALL', 'D_NP3RTALJ',
       'D_NP3RTCON', 'D_NP3TOT', 'D_NHY','PREV_SIGFALL','SIGFALL']]
df_qs = df_qs.dropna()
df_qs.to_csv('../working_data/updrsiii_all_q_p_s.csv', index=False)

df_q = df_qs.drop('PREV_SIGFALL', axis=1)
df_q.to_csv('../working_data/updrsiii_all_q_s.csv', index=False)

In [8]:
# Extract used PATNOS for population analysis
df_pop = df_updrs_iii_sigfall_base
df_pop = df_pop.dropna()
df_pop = df_pop[['PATNO', 'EVENT_ID', 'INFODT']]
df_pop.to_csv('../source_data/u_iii_records.csv')